# Projet AirBnB - Prétraitement des Données (Side Quest)
### Ce notebook a pour objectif de préparer le dataset pour la prédiction du prix d'un logement sur Airbnb.
### Il est structuré en plusieurs sections : chargement et exploration des données, prétraitement, et séparation des données pour une future modélisation.
### Chaque membre pourra compléter sa partie en fonction de son rôle.

 ## Table des Matières
### 1. Introduction
### 2. Chargement et Exploration des Données
### 3. Prétraitement des Données
###    - 3.1 Sélection des Colonnes Pertinentes
###    - 3.2 Suppression des Doublons
###    - 3.3 Gestion des Valeurs Manquantes
###    - 3.4 Conversion des Types de Données
###    - 3.5 Gestion des Valeurs Aberrantes
### 4. Séparation des Données (Train/Test Split)
### 5. Bonus: Préparation pour la Régression Linéaire
### 6. Conclusion et Perspectives


## 1. Introduction
### Dans ce projet, nous préparons le dataset AirBnB afin de prédire le prix des logements.
### Ce notebook se concentre sur la partie prétraitement des données avant l'application des algorithmes de prédiction.
### Chaque membre de l'équipe pourra ajouter des analyses ou des ajustements au fur et à mesure.


In [2]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import sys
print(sys.executable)

c:\Users\FranckPERSONNE\anaconda3\python.exe


 ## 2. Chargement et Exploration des Données
### Charger le fichier CSV et effectuer une première exploration du dataset.

In [4]:
# TODO: Remplacer le chemin par le chemin de votre fichier CSV si nécessaire
file_path = 'listings.csv'
df = pd.read_csv(file_path)

# Affichage des premières lignes du dataset
print(df.head())

# Informations générales sur le dataset
print(df.info())

# Statistiques descriptives
print(df.describe())


     id                        listing_url       scrape_id last_scraped  \
0  3109  https://www.airbnb.com/rooms/3109  20241206031559   2024-12-07   
1  5396  https://www.airbnb.com/rooms/5396  20241206031559   2024-12-09   
2  7397  https://www.airbnb.com/rooms/7397  20241206031559   2024-12-08   
3  7964  https://www.airbnb.com/rooms/7964  20241206031559   2024-12-09   
4  9359  https://www.airbnb.com/rooms/9359  20241206031559   2024-12-09   

            source                                             name  \
0      city scrape                                     zen and calm   
1      city scrape     Your perfect Paris studio on Île Saint-Louis   
2      city scrape                 MARAIS - 2ROOMS APT - 2/4 PEOPLE   
3  previous scrape                     Sunny apartment with balcony   
4      city scrape  Cozy, Central Paris: WALK or VELIB EVERYWHERE !   

                                         description  \
0  Lovely Appartment with one bedroom with a Quee...   
1  NEW SOF

 ## 3. Prétraitement des Données
### Dans cette section, nous allons préparer le dataset pour l'analyse.
### Cette partie est à compléter par les membres de l'équipe selon les besoins.

## 3.1 Sélection des Colonnes Pertinentes
### Sélectionner les colonnes jugées importantes pour la prédiction du prix.

In [6]:
# TODO: Adapter la liste des colonnes en fonction de vos besoins
colonnes_pertinentes = ['price', 'room_type', 'neighbourhood', 'minimum_nights', 'number_of_reviews']
df = df[colonnes_pertinentes]
# Vérification de la sélection
print(df.head())

     price        room_type            neighbourhood  minimum_nights  \
0  $100.00  Entire home/apt  Neighborhood highlights               2   
1   $88.00  Entire home/apt  Neighborhood highlights               3   
2  $136.00  Entire home/apt                      NaN              10   
3      NaN  Entire home/apt                      NaN               7   
4   $75.00  Entire home/apt                      NaN             180   

   number_of_reviews  
0                  4  
1                425  
2                367  
3                  5  
4                  0  


## 3.2 Suppression des Doublons
### Identifier et supprimer les doublons dans le dataset.

In [ ]:

df_cleaned = df.drop_duplicates(subset=['price', 'room_type', 'neighbourhood', 'minimum_nights', 'number_of_reviews'], keep='first')

print(df_cleaned)

print("Doublons avant suppression :", df.duplicated().sum())
print("Doublons après suppression :", df_cleaned.duplicated().sum())



         price        room_type            neighbourhood  minimum_nights  \
0      $100.00  Entire home/apt  Neighborhood highlights               2   
1       $88.00  Entire home/apt  Neighborhood highlights               3   
2      $136.00  Entire home/apt                      NaN              10   
3          NaN  Entire home/apt                      NaN               7   
4       $75.00  Entire home/apt                      NaN             180   
...        ...              ...                      ...             ...   
91004   $76.00  Entire home/apt                      NaN               7   
91007   $64.00  Entire home/apt  Neighborhood highlights               1   
91017   $56.00  Entire home/apt  Neighborhood highlights               1   
91028  $818.00  Entire home/apt  Neighborhood highlights               1   
91029  $840.00  Entire home/apt  Neighborhood highlights               1   

       number_of_reviews  
0                      4  
1                    425  
2     

## 3.3 Gestion des Valeurs Manquantes
### Identifier et traiter les valeurs manquantes.


In [ ]:

df_cleaned = df_cleaned.copy()

df_cleaned['price'] = df_cleaned['price'].replace({r'\$': '', r',': ''}, regex=True).astype(float)

df_cleaned['price'] = df_cleaned['price'].fillna(df_cleaned['price'].mean())

df_cleaned['neighbourhood'].fillna('Pas de neighbourhood', inplace=True)

print(df_cleaned.isnull().sum())


price                0
room_type            0
neighbourhood        0
minimum_nights       0
number_of_reviews    0
dtype: int64


C:\Users\FranckPERSONNE\AppData\Local\Temp\ipykernel_1524\3897750278.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['neighbourhood'].fillna('Pas de neighbourhood', inplace=True)


 # 3.4 Conversion des Types de Données
### Convertir les colonnes en types appropriés, par exemple convertir 'price' en numérique.

# 3.5 Gestion des Valeurs Aberrantes
### Détecter et gérer les valeurs aberrantes à l'aide de visualisations.

 # 4. Séparation des Données (Train/Test Split)
### Préparer le dataset pour la modélisation en séparant les données en ensemble d'entraînement et de test.


# 5. Bonus: Préparation pour la Régression Linéaire
### Placeholder pour la future implémentation d'une régression linéaire.
### Cette section pourra être complétée ultérieurement avec la modélisation et l'évaluation.

# 6. Conclusion et Perspectives

### Ce notebook constitue la base du prétraitement des données pour le projet AirBnB.
### Chaque membre pourra ajuster ou compléter les sections selon l'évolution du projet.
### Pensez à ajouter des commentaires et des visualisations pertinentes pour documenter vos choix.